# Label transfer

In [1]:
rm(list=ls())
# Set the working directory to where folders named after the samples are located. 
# The folder contains spliced.mtx, unspliced.mtx, barcodes and gene id files, and json files produced by scKB that documents the sequencing stats. 
setwd("/scratch/AG_Ohler/CheWei/scKB")

In [2]:
# Load libraries
suppressMessages(library(Seurat))

In [3]:
sessionInfo()

R version 3.6.1 (2019-07-05)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /fast/home/c/chsu/anaconda3/envs/seu314/lib/libopenblasp-r0.3.9.so

locale:
 [1] LC_CTYPE=en_US.utf-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.utf-8        LC_COLLATE=en_US.utf-8    
 [5] LC_MONETARY=en_US.utf-8    LC_MESSAGES=en_US.utf-8   
 [7] LC_PAPER=en_US.utf-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.utf-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] Seurat_3.1.5

loaded via a namespace (and not attached):
 [1] httr_1.4.1         tidyr_1.1.0        jsonlite_1.6.1     viridisLite_0.3.0 
 [5] splines_3.6.1      leiden_0.3.3       ggrepel_0.8.2      globals_0.12.5    
 [9] pillar_1.4.4       lattice_0.20-41    glue_1.4.1         reticula

In [5]:
rc.integrated <- readRDS('./Root_Atlas.rds')

In [6]:
# Function for reading in Seurat object
read_seu <- function(dir) { 
  seu <- readRDS(dir) 
  return(seu)
}

In [7]:
# Function for label transfer
label_transfer_init <- function(sample.name){
    # Read Seurat object
    seu <-read_seu(dir = paste0("./supp_data/COPILOT_RDS/",sample.name,"_COPILOT.rds"))
    # Find transfer anchors
    lt.anchors <- FindTransferAnchors(reference = rc.integrated, query = seu, normalization.method = "SCT", npcs = 50 ,dims = 1:50)
    # Transfer the data "time.celltype.anno"
    predictions <- TransferData(anchorset = lt.anchors, refdata = rc.integrated$time.celltype.anno, dims = 1:50, weight.reduction = "pcaproject")
    seu <- AddMetaData(seu, metadata = predictions)
    seu@meta.data$time.celltype.anno <- seu@meta.data$predicted.id
    # Transfer the data "celltype.anno"
    predictions <- TransferData(anchorset = lt.anchors, refdata = rc.integrated$celltype.anno, dims = 1:50, weight.reduction = "pcaproject")
    seu <- AddMetaData(seu, metadata = predictions)
    seu@meta.data$celltype.anno <- seu@meta.data$predicted.id
    # Transfer the data "time.anno"
    predictions <- TransferData(anchorset = lt.anchors, refdata = rc.integrated$time.anno, dims = 1:50, weight.reduction = "pcaproject")
    seu <- AddMetaData(seu, metadata = predictions)
    seu@meta.data$time.anno <- seu@meta.data$predicted.id
    # Transfer the data "consensus.time.group"
    predictions <- TransferData(anchorset = lt.anchors, refdata = rc.integrated$consensus.time.group, dims = 1:50, weight.reduction = "pcaproject")
    seu <- AddMetaData(seu, metadata = predictions)
    seu@meta.data$consensus.time.group <- seu@meta.data$predicted.id
    # Save Seurat object
    saveRDS(seu, file = paste0("./supp_data/COPILOT_RDS/",sample.name,"_COPILOT.rds"))
    # Save Anchors
    save(lt.anchors, file=paste0("./supp_data/COPILOT_RDS/",sample.name,"_lt_anchors.RData"))
}

In [8]:
# List of sample which labels are going to transferred from atlas
sample.list <- c("minus", "plus", "sc_17", "sc_18", "sc_19", "sc_28", "sc_29", "sc_35", "sc_55", "sc_56", "sc_57", "sc_58", "sc_59", "sc_60", "sc_61", "sc_62", "sc_63", "sc_64", "sc_66")

In [9]:
# Run label transfer
for (i in 1:length(sample.list)){
 label_transfer_init(sample.list[i])
}